# Text Extract from pdf

In [1]:
from src.data_prep import load_pdf_data_from_disk
text = load_pdf_data_from_disk(file_name='Inspira Psych Handbook 2023 -2024 edited.pdf')

In [2]:
text[1000:2000]

' ……………………. …………………………………………………………………………….. 59 \n \n \n \n \n \n \n \n \n \n \n \n Inspira Health Psychiatry Residency Handbook  \n \n3 \n  \nWelcome ! \nCongratulations on starting your Psychiatry Residency with us at Inspira Health! All your hard work \nleading to this point has paid off and the next phase of your journey is now just beginning. These next \nfew years will be your “formative years” in your pro fessional career,  and we are ecstatic to be a part of \nit. This handbook will serve as a guide on many different topics including: overarching goals, rotation \nspecifics, program logistics, professionalism, amongst others.  \nPlease take your time to review ou r policies and guidelines. You will be best prepared to start residency \nwhen you understand what the expectations are as a trainee and are for faculty. During the course of \nthe year, supplemental information may also be given to you. Please be aware that a ny policy in this \nmanual is subject to change.  \nFor furt

In [3]:
from src.data_prep import clean_and_format_text
clean_text = clean_and_format_text(text)

In [4]:
clean_text[1000:3000]

'ychiatry Residency Handbook 3 Welcome! Congratulations on starting your Psychiatry Residency with us at Inspira Health! All your hard work leading to this point has paid off and the next phase of your journey is now just beginning. These next few years will be your “formative years” in your pro fessional career, and we are ecstatic to beapart of it. This handbook will serve asaguide on many different topics including: overarching goals, rotation specifics, program logistics, professionalism, amongst others. Please take your time to review ourpolicies and guidelines. You will be best prepared to start residency when you understand what the expectations are as a trainee and are for faculty. During the course of the year, supplemental information may also be given to you. Please be aware thatany policy in this manual is subject to change. For further clarification of clinical information and/or service requirements, please speak with your Program Director, the director of the individual 

In [5]:
# Save the cleaned text to a text file
with open("data/clean_pdf_text", 'w') as file:
    file.write(clean_text)

In [ ]:
from flask import Flask, render_template_string, request, jsonify
import requests

from src.data_prep import load_pdf_data_from_disk
text = load_pdf_data_from_disk(file_name='Inspira Psych Handbook 2023 -2024 edited.pdf')

from src.data_prep import clean_and_format_text
handbook_text = clean_and_format_text(text)

app = Flask(__name__)

# Dummy API key for demonstration. Replace with your actual API key when running locally.
API_KEY = "HIDDEN"
OPENAI_ENDPOINT = "https://api.openai.com/v1/engines/davinci/completions"
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "User-Agent": "Inspira-Psych-Handbook-App"
}

template_string = """
<!doctype html>
<html>
    <head>
        <title>Student Handbook Q&A</title>
    </head>
    <body>
        <h2>Ask a Question About the Student Handbook</h2>
        <form action="/ask_openai" method="post" id="questionForm">
            <textarea name="question" rows="4" cols="50" placeholder="Type your question here..."></textarea><br><br>
            <input type="submit" value="Ask">
        </form>
        <h3>Answer:</h3>
        <div id="answer"></div>
        <script>
            const form = document.getElementById('questionForm');
            form.addEventListener('submit', async (event) => {
                event.preventDefault();
                const formData = new FormData(form);
                const response = await fetch('/ask_openai', {
                    method: 'POST',
                    body: formData
                });
                const data = await response.json();
                document.getElementById('answer').textContent = data.answer;
            });
        </script>
    </body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(template_string)

@app.route('/ask_openai', methods=['POST'])
def ask_openai():
    question = request.form.get('question')
    prompt = f"The following is a student handbook: {handbook_text}\n\nQ: {question}\nA:"
    data = {
        "prompt": prompt,
        "max_tokens": 250
    }
    response = requests.post(OPENAI_ENDPOINT, headers=headers, json=data)
    response_data = response.json()
    print(response_data)  # This will log the entire response to your console.
    answer = response_data['choices'][0]['text'].strip() if 'choices' in response_data and response_data['choices'] else "Sorry, I couldn't provide an answer at the moment."
    return jsonify({"answer": answer})

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Jul/2023 17:28:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2023 17:28:42] "POST /ask_openai HTTP/1.1" 200 -


{'error': {'message': "This model's maximum context length is 2049 tokens, however you requested 22359 tokens (22109 in your prompt; 250 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
